# 模型比较

In [17]:
# 导入相关包
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


In [18]:
dataset = pd.read_csv("data.csv", encoding="gbk")

In [19]:
# 预处理
# 每单平均交易量
dataset['per_avg_amount'] = dataset['historical_trans_amount'] / dataset['number_of_trans_from_2011']
# 每天平均交易量
dataset['avg_amount_perday'] = dataset['historical_trans_amount'] / dataset['historical_trans_day']

#  可以删掉的列
delete_cols = ["Unnamed: 0", "custid", "trade_no", "bank_card_no", "source", "id_name", "student_feature",
              "query_finance_count", "query_cash_count", "query_org_count", # 保留一个总的即可
               "historical_trans_amount", "number_of_trans_from_2011", "historical_trans_amount", "historical_trans_day"
               
              ]
dataset.drop(delete_cols, axis=1, inplace=True)

In [20]:
dataset.head()

,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,repayment_capability,is_high_user,first_transaction_time,rank_trad_1_month,trans_amount_3_month,avg_consume_less_12_valid_month,abs,top_trans_count_last_1_month,avg_price_last_12_month,avg_price_top_last_12_valid_month,reg_preference_for_trad,trans_top_time_last_1_month,trans_top_time_last_6_month,consume_top_time_last_1_month,consume_top_time_last_6_month,cross_consume_count_last_1_month,trans_fail_top_count_enum_last_1_month,trans_fail_top_count_enum_last_6_month,trans_fail_top_count_enum_last_12_month,consume_mini_time_last_1_month,max_cumulative_consume_later_1_month,max_consume_count_later_6_month,railway_consume_count_last_12_month,pawns_auctions_trusts_consume_last_1_month,pawns_auctions_trusts_consume_last_6_month,jewelry_consume_count_last_6_month,status,first_transaction_day,trans_day_last_12_month,apply_score,apply_credibility,query_sum_count,latest_query_time,latest_one_month_apply,latest_three_month_apply,latest_six_month_apply,loans_score,loans_credibility_behavior,loans_count,loans_settle_count,loans_overdue_count,loans_org_count_behavior,consfin_org_count_behavior,loans_cash_count,latest_one_month_loan,latest_three_month_loan,latest_six_month_loan,history_suc_fee,history_fail_fee,latest_one_month_suc,latest_one_month_fail,loans_long_time,loans_latest_time,loans_credit_limit,loans_credibility_limit,loans_org_count_current,loans_product_count,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day,per_avg_amount,avg_amount_perday
0,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,26.0,3.0,19890,0,20130817.0,0.40,34030,7.0,3920,0.15,1020,0.55,一线城市,4.0,19.0,4.0,19.0,1.0,1.0,2.0,2.0,5.0,2170,6.0,0.0,1970,18040,0.0,1,1738.0,85.0,583.0,79.0,10.0,2018-04-25,2.0,5.0,8.0,552.0,73.0,37.0,34.0,2.0,10.0,1.0,9.0,1.0,1.0,13.0,37.0,7.0,1.0,0.0,341.0,2018-04-19,2200.0,72.0,9.0,10.0,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0,4968.333333,987.086093
1,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,14.0,4.0,16970,0,20160402.0,0.35,10590,5.0,6950,0.05,1210,0.50,一线城市,13.0,30.0,13.0,30.0,0.0,0.0,3.0,3.0,330.0,2100,9.0,0.0,1820,15680,0.0,0,779.0,84.0,653.0,73.0,8.0,2018-05-03,2.0,6.0,8.0,635.0,76.0,37.0,36.0,0.0,17.0,5.0,12.0,2.0,2.0,8.0,49.0,4.0,2.0,1.0,353.0,2018-05-05,2000.0,74.0,12.0,12.0,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0,13170.000000,1352.276786
2,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,22.0,1.0,9710,0,20170617.0,1.00,5710,5.0,840,0.65,570,0.65,一线城市,0.0,68.0,0.0,68.0,0.0,3.0,6.0,6.0,0.0,0,3.0,0.0,0,0,0.0,1,338.0,95.0,654.0,76.0,16.0,2018-05-05,5.0,5.0,14.0,633.0,83.0,4.0,2.0,0.0,3.0,1.0,2.0,2.0,2.0,4.0,2.0,2.0,1.0,1.0,157.0,2018-05-01,1500.0,77.0,2.0,2.0,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0,1280.000000,371.612903
3,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,6.0,3.0,6210,0,20130516.0,0.15,91690,7.0,46850,0.05,1290,0.45,三线城市,6.0,8.0,6.0,8.0,0.0,1.0,8.0,8.0,31700.0,8140,9.0,0.0,2700,27970,0.0,0,1831.0,82.0,595.0,79.0,22.0,2018-05-05,3.0,16.0,17.0,542.0,75.0,85.0,81.0,4.0,22.0,5.0,17.0,2.0,4.0,34.0,91.0,26.0,2.0,0.0,355.0,2018-05-03,1800.0,74.0,17.0,18.0,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0,14882.727273,1364.250000
4,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,42.0,1.0,11150,0,20170312.0,0.65,9770,6.0,760,1.00,1110,0.50,一线城市,0.0,66.0,0.0,66.0,0.0,3.0,3.0,3.0,0.0,1000,3.0,0.0,0,6410,0.0,1,435.0,88.0,541.0,75.0,14.0,2018-04-15,6.0,8.0,9.0,479.0,73.0,37.0,32.0,6.0,12.0,2.0,10.0,0.0,0.0,10.0,36.0,25.0,0.0,0.0,360.0,2018-01-07,1800.0,72.0,10.0,10.0,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0,5122.500000,975.714286


In [21]:
# 创建测试集 ，为了防止采样偏差，这里使用分层采样
# 测试集通常被忽略，但实际是机器学习非常重要的一部分。
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=2018)
for train_index, test_index in split.split(dataset, dataset["status"]):
    strat_train_set=dataset.loc[train_index]
    strat_test_set=dataset.loc[test_index]

# 分层采样就是按照这里给的 status 分类比例来的
dataset["status"].value_counts()/len(dataset)

0    0.749053
1    0.250947
Name: status, dtype: float64

In [22]:
train = strat_train_set.copy()
corr_matrix = train.corr()
selected_columns = list(corr_matrix['status'].sort_values(ascending = False)[:20].index.values)
X_train= train[selected_columns].drop(["status"], axis=1)
y_train= train[["status"]]

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


def transform(df):
    """ 
        特征转换
    """
    # 类别属性
    # cat_columns = ['reg_preference_for_trad', 'latest_query_time', 'loans_latest_time']
    cat_columns = []
    # 数值属性
    num_cols = X_train.columns
    steps = [
        ('imputer', SimpleImputer(strategy="median")),
        ('ss', StandardScaler())]
    num_pipe = Pipeline(steps)
    transformers = [
        ('num', num_pipe, num_cols),
        ("cat", OneHotEncoder(), cat_columns),
    ]
    ct = ColumnTransformer(transformers=transformers)
    X_num_transformed = ct.fit_transform(df)
    return X_num_transformed
X = transform(X_train)

In [35]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression  # 逻辑回归
from sklearn.tree import DecisionTreeClassifier # 决策树
from sklearn.svm import SVC # SVM 
from sklearn.ensemble import RandomForestClassifier # 随机森林
from xgboost import XGBClassifier # XGB

models = [
    ('LR'   , LogisticRegression()),
    ('CART' , DecisionTreeClassifier()),
    ('SVM'  , SVC(probability=True)),
    ('RF'   , RandomForestClassifier()),
    ('XGB'   , XGBClassifier())
]

# evaluates = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']


def run_models(x, y, models):
    num_folds = 10
    scoring = 'accuracy'

    results = []
    names = []
    for name, model in models:
        model.fit(x,y)
        kfold = StratifiedKFold(n_splits=num_folds, random_state=123)
        cv_results = cross_val_score(model, x, y, cv=kfold, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
    return names, results

In [36]:
# 运行模型
names, results = run_models(X, y_train, models)

LR: 0.787810 (0.011398)
CART: 0.696690 (0.031717)
SVM: 0.785098 (0.008241)
RF: 0.777284 (0.017330)
XGB: 0.788393 (0.010773)


In [44]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印绘制模型结果
def model_metrics(clf, X_train, X_test, y_train, y_test):
    # 预测
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    y_train_proba = clf.predict_proba(X_train)[:, 1]
    y_test_proba = clf.predict_proba(X_test)[:, 1]

    # 准确率
    print('[准确率]', end=' ')
    print('训练集：', '%.4f' % accuracy_score(y_train, y_train_pred), end=' ')
    print('测试集：', '%.4f' % accuracy_score(y_test, y_test_pred))

    # 精准率
    print('[精准率]', end=' ')
    print('训练集：', '%.4f' % precision_score(y_train, y_train_pred), end=' ')
    print('测试集：', '%.4f' % precision_score(y_test, y_test_pred))

    # 召回率
    print('[召回率]', end=' ')
    print('训练集：', '%.4f' % recall_score(y_train, y_train_pred), end=' ')
    print('测试集：', '%.4f' % recall_score(y_test, y_test_pred))

    # f1-score
    print('[f1-score]', end=' ')
    print('训练集：', '%.4f' % f1_score(y_train, y_train_pred), end=' ')
    print('测试集：', '%.4f' % f1_score(y_test, y_test_pred))

    # auc取值：用roc_auc_score或auc
    print('[auc值]', end=' ')
    print('训练集：', '%.4f' % roc_auc_score(y_train, y_train_proba), end=' ')
    print('测试集：', '%.4f' % roc_auc_score(y_test, y_test_proba))

    # roc曲线
    fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_proba, pos_label=1)
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_test_proba, pos_label=1)

    label = ["Train - AUC:{:.4f}".format(auc(fpr_train, tpr_train)),
             "Test - AUC:{:.4f}".format(auc(fpr_test, tpr_test))]
    plt.plot(fpr_train, tpr_train)
    plt.plot(fpr_test, tpr_test)
    plt.plot([0, 1], [0, 1], 'd--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(label, loc=4)
    plt.title("ROC curve")
    plt.show()

X_test= strat_test_set[selected_columns].drop(["status"], axis=1)
X_test = transform(X_test)
y_test= strat_test_set[["status"]]
    
for name, model in models:
    model_metrics(model, X, X_test, y_train, y_test)


[准确率] 训练集： 0.7914 测试集： 0.7905
[精准率] 训练集： 0.6880 测试集： 0.6855
[召回率] 训练集： 0.3090 测试集： 0.3045
[f1-score] 训练集： 0.4264 测试集： 0.4217
[auc值] 训练集： 0.7938 测试集： 0.7715


NameError: name 'roc_curve' is not defined